## Finding Your Way In The City (Graph Edition)
In this notebook your attention will shift from grids to graphs. At least for search ... 

Using Voronoi graphs and the medial axis transform we can find paths which maximize safety from obstacles. In addition, graph representation allows further optimizations and more succinct queries.

In [1]:
# OK this might look a little ugly but...
# need to import the latest version of networkx
# This occassionally fails, so if the next cell 
# doesn't show that you're using networkx 2.1
# please "restart and clear output" from "Kernel" menu
# above and try again.
import sys
!{sys.executable} -m pip install -I networkx==2.1
# import pkg_resources
# pkg_resources.require("networkx==2.1")

Processing /Users/michaelrose/Library/Caches/pip/wheels/73/23/af/0b183d0c311bbd174d61613a506bc22c7134eca8878d09aec0/networkx-2.1-py2.py3-none-any.whl
  Using cached decorator-4.4.1-py2.py3-none-any.whl (9.2 kB)


In [2]:
import networkx as nx
nx.__version__

'2.1'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as LA

import sys
sys.path.append('..')
from grid import create_grid_and_edges

%matplotlib inline 

ImportError: cannot import name 'create_grid_and_edges'

In [ ]:
plt.rcParams['figure.figsize'] = 12, 12

In [ ]:
# This is the same obstacle data from the previous lesson.
filename = '../colliders.csv'
data = np.loadtxt(filename, delimiter=',', dtype='Float64', skiprows=2)
print(data)

Starting and goal positions in *(north, east)*.

In [ ]:
start_ne = (25,  100)
goal_ne = (750., 370.)

In [ ]:
# Static drone altitude (metres)
drone_altitude = 5
safety_distance = 3

In [ ]:
# This is now the routine using Voronoi
grid, edges = create_grid_and_edges(data, drone_altitude, safety_distance)
print(len(edges))

Plot the edges on top of the grid along with start and goal locations.

In [ ]:
# equivalent to
# plt.imshow(np.flip(grid, 0))
plt.imshow(grid, origin='lower', cmap='Greys') 

for e in edges:
    p1 = e[0]
    p2 = e[1]
    plt.plot([p1[1], p2[1]], [p1[0], p2[0]], 'b-')

    
plt.plot(start_ne[1], start_ne[0], 'rx')
plt.plot(goal_ne[1], goal_ne[0], 'rx')

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()

We now have a graph, well at least visually. The next step is to use the [`networkx`](https://networkx.github.io) to create the graph. **NetworkX** is a popular library handling anything and everything related to graph data structures and algorithms.

**NOTE:** In the initial import above it was imported with the `nx` alias.

You're encouraged to read the documentation but here's a super quick tour:

1. Create a graph:

```
G = nx.Graph()
```

2. Add an edge:

```
p1 = (10, 2.2)
p2 = (50, 40)
G = nx.add_edge(p1, p2)
```

3 Add an edge with a weight:

```
p1 = (10, 2.2)
p2 = (50, 40)
dist = LA.norm(np.array(p2) - np.array(p1))
G = nx.add_edge(p1, p2, weight=dist)
```

In [ ]:
# TODO: create the graph with the weight of the edges
# set to the Euclidean distance between the points
G = nx.Graph()
for p1, p2 in edges:
    dist = LA.norm(np.array(p2) - np.array(p1))
    G.add_edge(p1, p2, weight=dist)

for n in G.nodes:
    print('Node:', n)
    print(G[n])
    print('Neighbors:')
    for a in G.neighbors(n):
        print(a)
        print(G.get_edge_data(n, a))
    print('----')   

You need a method to search the graph, and you'll adapt A* in order to do this. The notable differences being the actions are now the outgoing edges and the cost of an action is that weight of that edge.

In [ ]:
from queue import PriorityQueue

def heuristic(n1, n2):
    return LA.norm(np.array(n1) - np.array(n2))

###### THIS IS YOUR OLD GRID-BASED A* IMPLEMENTATION #######
###### With a few minor modifications it can work with graphs! ####
#TODO: modify A* to work with a graph
def a_star(graph, h, start, goal):

    path = []
    path_cost = 0
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)

    branch = {}
    found = False
    
    while not queue.empty():
        item = queue.get()
        current_node = item[1]
        if current_node == start:
            current_cost = 0.0
        else:              
            current_cost = branch[current_node][0]
            
        if current_node == goal:        
            print('Found a path.')
            found = True
            break
        else:
            for next_node in graph.neighbors(current_node):
                branch_cost = current_cost + graph.get_edge_data(current_node,next_node)['weight']
                queue_cost = branch_cost + h(next_node, goal)
                
                if next_node not in visited:                
                    visited.add(next_node)               
                    branch[next_node] = (branch_cost, current_node)
                    queue.put((queue_cost, next_node))
             
    if found:
        # retrace steps
        n = goal
        path_cost = branch[n][0]
        path.append(goal)
        while branch[n][1] != start:
            path.append(branch[n][1])
            n = branch[n][1]
        path.append(branch[n][1])
    else:
        print('**********************')
        print('Failed to find a path!')
        print('**********************') 
    return path[::-1], path_cost


### Solution

This solution consists of two parts:

1. Find the closest point in the graph to our current location, same thing for the goal location.
2. Compute the path from the two points in the graph using the A* algorithm.
3. Feel free to use any of the path pruning techniques to make the path even smaller! 
4. Plot it up to see the results!

### TODO: Write your solution!

[our solution](/notebooks/Graph-Search-Solution.ipynb)

In [ ]:
def find_nearest_node(graph, pos):
    # Find the nearest node on the graph to the
    # specified position
    
    # Generate an array of all node positions
    nodes = np.array(graph.nodes)

    # Calculate the distance of all nodes to the specified position
    dist = LA.norm(nodes - np.array(pos), axis=1)
    # Find the index of the minimum distance element and get the
    # node position
    near_node = nodes[dist.argmin()]
    
    return tuple(near_node)

graph_start = find_nearest_node(G, start_ne)
graph_goal = find_nearest_node(G, goal_ne)

print(start_ne, goal_ne)
print(graph_start, graph_goal)


In [ ]:
# 1a. Find the closest point in the graph to the start location:
graph_start = find_nearest_node(G, start_ne)

# 1b. Find the closest point in the graph to the goal location:
graph_goal = find_nearest_node(G, goal_ne)

# 2. Compute the path from the two points in the graph using the A* algorithm.
path, path_cost = a_star(G, heuristic, graph_start, graph_goal)
print("Path found of length {0}. Total cost: {1}".format(len(path), path_cost))

# 3. Feel free to use any of the path pruning techniques to make the path even smaller! 
# TODO: Prune path using bresenham?

# 4. Plot it up to see the results!
plt.imshow(grid, cmap='Greys', origin='lower')

for e in edges:
    p1 = e[0]
    p2 = e[1]
    plt.plot([p1[1], p2[1]], [p1[0], p2[0]], 'b-')

# For the purposes of the visual the east coordinate lay along
# the x-axis and the north coordinates long the y-axis.
plt.plot(start_ne[1], start_ne[0], 'gx')
# Uncomment the following as needed
plt.plot(goal_ne[1], goal_ne[0], 'rx')

pp = np.array(path)
plt.plot(pp[:, 1], pp[:, 0], 'm')

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()